In [29]:
import pandas as pd

def extract_short_features(window_size, signals):

    print("extracting short features with window size " + str(window_size))
    short_features_with_headers = [feature_extraction(s, 44100, window_size, window_size) for s in signals]

    headers = short_features_with_headers[0][1]

    transposed = []
    for j in range(len(short_features_with_headers[0][0])):
        new_row = []
        for row in short_features_with_headers:
            new_row.append(row[0][j])
        transposed.append(new_row)

    return pd.DataFrame(transposed, index= headers).transpose(copy=False)


In [38]:
from numpy import linalg as la

def short_features_distances(short_features_df):

    distances_df = short_features_df.copy()

    for j in range(0, 68):
        for i in range(1, 8):
            minlen = min(len(distances_df.iloc[0, j]), len(distances_df.iloc[i, j]))
            distances_df.iloc[i, j] = la.norm(distances_df.iloc[0, j][:minlen] - distances_df.iloc[i, j][:minlen])
            
    distances_df.drop(0, inplace=True)
    return distances_df

In [42]:
from pyAudioAnalysis.pyAudioAnalysis.audioBasicIO import read_audio_file, stereo_to_mono
from pyAudioAnalysis.pyAudioAnalysis.ShortTermFeatures import feature_extraction
import scipy.stats as stats

signals = [stereo_to_mono(read_audio_file(f"data/{i}.mp3")[1]) for i in range(0, 8)]

short_1000 = short_features_distances(extract_short_features(1000, signals))
short_2000 = short_features_distances(extract_short_features(2000, signals))
short_5000 = short_features_distances(extract_short_features(5000, signals))
short_10000 = short_features_distances(extract_short_features(10000, signals))

extracting short features with window size 1000
extracting short features with window size 2000
extracting short features with window size 5000
extracting short features with window size 10000


In [ ]:
perfect = [1,2,3,4,5,6,7]
cors = pd.DataFrame(columns = ["correlation1000", "correlation2000", "correlation5000", "correlation10000"], index = short_1000.columns)

for col in cors.index:
    cors.loc[col, "correlation1000"] = stats.spearmanr(short_1000.loc[:,col], perfect)[1]
    cors.loc[col, "correlation2000"] = stats.spearmanr(short_2000.loc[:,col], perfect)[1]
    cors.loc[col, "correlation5000"] = stats.spearmanr(short_5000.loc[:,col], perfect)[1]
    cors.loc[col, "correlation10000"] = stats.spearmanr(short_10000.loc[:,col], perfect)[1]

cors

,correlation1000,correlation2000,correlation5000,correlation10000
zcr,0.013697,0.013697,0.013697,0.013697
energy,0.052181,0.052181,0.052181,0.023449
energy_entropy,0.25317,0.215217,0.09375,0.25317
spectral_centroid,0.013697,0.013697,0.013697,0.013697
spectral_spread,0.819151,0.819151,0.879048,0.644512
...,...,...,...,...
delta chroma_9,0.013697,0.013697,0.071344,0.383317
delta chroma_10,0.939408,0.215217,0.879048,0.534509
delta chroma_11,0.759945,0.644512,0.052181,0.023449
delta chroma_12,0.879048,0.644512,0.293934,0.759945
